In [1]:
import mne
import pandas as pd
import numpy as np
import glob
import pylab
import pandas as pd
import pickle
import matplotlib.pyplot as plt


stim_duration = 3
run_length = 4
first_run_file = 1
last_run_file = 4
S = 'S04'
bids_top_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/'
#proj_path = "/misc/data12/sjapee/Sebastian-OrientationImagery/Data/"
#raw_dir = proj_path + '20240611/' + S + '/'
#logfile_dir = raw_dir + 'sheets/extracted_dataMovie'
bids_dir = bids_top_dir + f'bids_dir/sub-{S}/ses-1/meg/'

In [2]:
#*****************************#
### PARAMETERS ###
#*****************************#

l_freq                      = 0.1
h_freq                      = 100
notch                       = 60
notch_max                   = 240
pre_stim_time               = -.2
post_stim_time              = 3.2
std_deviations_above_below  = 4
output_resolution           = 200
trigger_channel             = 'UPPT001'


import glob

dsets = []
for i in range(1, 5, 1):
    ds = f'run-{i:02}'  # Format the run number with leading zeros
    pattern = f'{bids_dir}*Dynamic_{ds}_meg.ds'  # Adjust the pattern to match your file name structure
    matches = glob.glob(pattern)
    
    if matches:
        dsets.append(matches[0])
    else:
        print(f"No files found for pattern: {pattern}")

print(dsets)



['/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-01_meg.ds', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-02_meg.ds', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-03_meg.ds', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-04_meg.ds']


In [3]:
#New way of defining onsets not based on standard deviation but based on finding photodiode after the trigger
def defineOnsets(raw, events):
    log_statements = []
    pd_dat = raw[raw.ch_names.index('UADC016-2104')][0][0]
    if np.mean(np.abs(pd_dat))>0.6: # this should be fine but may not be good if there are a looot of zeros
        # zero-centering & making everything positive
        pd_dat = np.abs(pd_dat-np.mean(pd_dat[0:100]))
        # scaling channel to 0-1
        pd_dat = (pd_dat-np.min(pd_dat)) / (np.max(pd_dat)-np.min(pd_dat))

        print(pd_dat)

        # using the event, we are now looking in the next 50ms of the photodiode channel (digital) when the signal goes above 0.5
        t_samples = int(0.05/(1/raw.info['sfreq']))
        threshold =  .5

        # when we found the events we calculate the trigger delay. 
        pd_events = [events[i,0]+np.where(pd_dat[events[i,0]:events[i,0]+t_samples]>threshold)[0][0] for i in range(len(events))]
        pd_onsets = []
        pd_offsets = []

        for event in events:
            # Find onset
            onset_index = event[0] + np.argmax(pd_dat[event[0]:event[0] + t_samples] > threshold)
            pd_onsets.append(onset_index)

            # Find offset
            pd_offset_index = onset_index + np.argmax(pd_dat[onset_index:] < threshold)
            pd_offsets.append(pd_offset_index)
        trigger_delay = 1000. * (pd_events-events[:, 0]) / raw.info['sfreq']
        # sometimes this channel records ON as UP and sometimes as DOWN. This if-loop makes sure we always find the onsets
        if np.mean(trigger_delay)==0:
            pd_events = [events[i,0]+np.where(pd_dat[events[i,0]:events[i,0]+t_samples]>threshold)[0][0] for i in range(len(events))]
            trigger_delay = 1000. * (pd_events-events[:, 0]) / raw.info['sfreq']
        log_statements.append(('Trigger delay removed (μ ± σ): %0.1f ± %0.1f ms')
            % (np.mean(trigger_delay), np.std(trigger_delay)))
        if np.mean(trigger_delay)>10:
            events[:,0] = events[:,0]+5
            log_statements.append(f'photo diode and parallel port triggers are too far apart. using parallel port trigger ')
        else:
            log_statements.append(f'photo diode used successfully ')
            events[:, 0] = pd_events
    else: 
        # if the pd channel didn't work, add 5 samples as a standard delay
        events[:,0] = events[:,0]+5
        log_statements.append(f'photo diode did not record anything. using parallel port trigger ')

    return raw,events,log_statements, pd_onsets, pd_offsets


def notch_filter(raw):
    #apply notch filter based on global parameters set at the top
    notches = np.arange(notch, notch_max+1, notch)
    print(notches)
    raw.notch_filter(notches, phase='zero-double', fir_design='firwin2')
    return raw
def prepMarkers():
    subs   = ['S01']
    block_conditions = {
        'Watch_Still': ['0022_Left', '0022_Right', '0067_Left', '0067_Right', '0112_Left', '0112_Right', 
                        '0157_Left', '0157_Right', '0202_Left', '0202_Right', '0247_Left', '0247_Right',
                        '0292_Left', '0292_Right', '0337_Left', '0337_Right']}
    trial_data = []
    blocks = list(block_conditions.keys())
    for block in blocks:
        conditions = block_conditions[block]
        for condition in conditions:
            block_data = {'Block': block, 'Condition': condition}
            trial_data.append(block_data)

    df = pd.DataFrame(trial_data)
    print(df)
    df['Code'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    ids = df['Code']
    conds = df['Block']
    orients = df['Condition']
    event_id = {}
    for i, id in enumerate(ids):
        event_id[id] = f'{subs[0]}/{conds[i]}/{orients[i]}'
    #keys = [f'{sub}/{cond}/{orient}/{id}' for sub in subs for id in ids for cond in conds for orient in orients]
    return event_id

In [4]:
event_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [5]:
from mne.preprocessing import ICA


all_epochs = []
all_events = []
all_onsets = []
all_offsets = []
for i in range(len(dsets)):
    raw = mne.io.read_raw_ctf(dsets[i],system_clock='ignore',preload=True)
    events = mne.find_events(raw, stim_channel='UPPT001',min_duration = 0.002)
    #print('events', events)
    #print('events2', events[-1])
    raw, events, log_statements, onsets, offsets = defineOnsets(raw, events)
    # remove trailing zeros
    endtime = raw.times[events[-1,0]]+10
    print(endtime)
    raw.crop(0,endtime)
    raw = notch_filter(raw)
    #ICA thing 
    epochs = mne.Epochs(raw, events, event_id=event_id,tmin=pre_stim_time, tmax=post_stim_time, preload=True, baseline=(None,0), picks = 'mag')
    print("Number of events:", len(events))
    print("Number of epochs:", len(epochs))
    print(np.nonzero(list(map(len, epochs.drop_log)))[0])
    [n for n, dl in enumerate(epochs.drop_log) if len(dl)]
    all_epochs.append(epochs)
    all_events.append(events)
    all_onsets.append(onsets)
    all_offsets.append(offsets)

print(len(all_events))

ds directory : /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       1.59   64.12    0.00 mm <->    1.59   64.12    0.00 mm (orig :  -38.22   57.08 -251.90 mm) diff =    0.000 mm
      -1.59  -64.12    0.00 mm <->   -1.59  -64.12    0.00 mm (orig :   60.55  -24.42 -259.45 mm) diff =    0.000 mm
      75.88    0.00    0.00 mm <->   75.88    0.00   -0.00 mm (orig :   55.17   69.72 -224.53 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImager

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    2.2s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    3.7s


Not setting metadata
122 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 122 events and 4081 original time points ...
0 bad epochs dropped
Number of events: 122
Number of epochs: 122
[]
ds directory : /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-02_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       1.86   64.16    0.00 mm <->    1.86   64.16   -0.00 mm (orig :  -38.04   57.56 -251.98 mm) diff =    0.000 mm
      -1.86  -64.16    0.00 mm <->   -1.86  -64.16   -0.00 mm (orig :   61.68  -22.89 -260.02 mm) diff =    0.000 mm
      78.63    0.00    0.00 mm <->   78.63   -0.00    0

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    2.5s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    4.1s


Not setting metadata
122 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 122 events and 4081 original time points ...
0 bad epochs dropped
Number of events: 122
Number of epochs: 122
[]
ds directory : /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-03_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       1.79   64.25    0.00 mm <->    1.79   64.25    0.00 mm (orig :  -37.83   57.87 -251.95 mm) diff =    0.000 mm
      -1.79  -64.25    0.00 mm <->   -1.79  -64.25    0.00 mm (orig :   62.54  -22.02 -260.17 mm) diff =    0.000 mm
      79.48    0.00    0.00 mm <->   79.48   -0.00    0

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.5s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    2.9s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    4.6s


Not setting metadata
122 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 122 events and 4081 original time points ...
0 bad epochs dropped
Number of events: 122
Number of epochs: 122
[]
ds directory : /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-04_meg.ds
    res4 data read.
    hc data not present
    Separate EEG position data file not present.
    Coordinate transformations established.
    Measurement info composed.
Finding samples for /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-04_meg.ds/sub-S04_ses-1_task-OrientationImageryDynamic_run-04_meg.meg4: 
    System 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.6s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    3.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    4.7s


Not setting metadata
122 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 122 events and 4081 original time points ...
0 bad epochs dropped
Number of events: 122
Number of epochs: 122
[]
4


In [6]:
pd_onsets_np = np.array(all_onsets)
pd_offsets_np = np.array(all_offsets)
offsets_minus_onsets_np = pd_offsets_np - pd_onsets_np
print(offsets_minus_onsets_np)
np.unique(offsets_minus_onsets_np)

[[3620 3619 3619 3620 3619 3619 3619 3620 3619 3619 3619 3619 3619 3620
  3619 3619 3619 3620 3619 3619 3620 3619 3619 3620 3619 3619 3619 3600
  3619 3619 3620 3619 3619 3620 3619 3619 3619 3620 3619 3619 3620 3619
  3619 3620 3619 3619 3620 3619 3619 3619 3620 3619 3619 3620 3619 3619
  3620 3619 3619 3620 3619 3619 3619 3620 3619 3619 3620 3619 3619 3619
  3620 3619 3619 3620 3619 3619 3619 3619 3619 3620 3619 3619 3620 3619
  3619 3619 3620 3619 3619 3620 3619 3619 3620 3619 3619 3619 3620 3619
  3620 3619 3619 3619 3619 3619 3619 3620 3619 3619 3619 3619 3619 3620
  3619 3619 3620 3619 3619 3619 3620 3619 3619 3620]
 [3620 3619 3619 3620 3619 3619 3619 3620 3619 3619 3619 3619 3620 3619
  3619 3620 3619 3619 3619 3620 3619 3620 3619 3619 3619 3620 3619 3619
  3620 3619 3619 3619 3619 3619 3619 3620 3619 3619 3620 3619 3619 3619
  3620 3619 3619 3620 3619 3619 3620 3619 3619 3619 3620 3619 3620 3619
  3619 3619 3620 3619 3619 3620 3619 3619 3619 3619 3619 3619 3620 3619
  3619 3620

array([3600, 3619, 3620])

In [40]:
print(len(all_epochs))
print(all_epochs)

4
[<Epochs |  122 events (all good), -0.2 – 3.2 s, baseline -0.2 – 0 s, ~1022.0 MB, data loaded,
 '1': 7
 '2': 7
 '3': 7
 '4': 7
 '5': 7
 '6': 7
 '7': 7
 '8': 7
 '9': 7
 '10': 7
 and 7 more events ...>, <Epochs |  122 events (all good), -0.2 – 3.2 s, baseline -0.2 – 0 s, ~1022.0 MB, data loaded,
 '1': 7
 '2': 7
 '3': 7
 '4': 7
 '5': 7
 '6': 7
 '7': 7
 '8': 7
 '9': 7
 '10': 7
 and 7 more events ...>, <Epochs |  122 events (all good), -0.2 – 3.2 s, baseline -0.2 – 0 s, ~1022.0 MB, data loaded,
 '1': 7
 '2': 7
 '3': 7
 '4': 7
 '5': 7
 '6': 7
 '7': 7
 '8': 7
 '9': 7
 '10': 7
 and 7 more events ...>, <Epochs |  122 events (all good), -0.2 – 3.2 s, baseline -0.2 – 0 s, ~1022.0 MB, data loaded,
 '1': 7
 '2': 7
 '3': 7
 '4': 7
 '5': 7
 '6': 7
 '7': 7
 '8': 7
 '9': 7
 '10': 7
 and 7 more events ...>]


In [7]:
dev_head_t_ref = all_epochs[0].info['dev_head_t']

for i in range(0, 4):
    all_epochs[i].info['dev_head_t'] = dev_head_t_ref

epochs_stacked = mne.concatenate_epochs(all_epochs)

/var/folders/9m/6b08zp213znc3k12g6gpjj9rqx2ly9/T/ipykernel_9955/987737906.py:6: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_stacked = mne.concatenate_epochs(all_epochs)


Not setting metadata
488 matching events found
Applying baseline correction (mode: mean)


In [8]:
pattern = f'{bids_dir}*events.tsv'  # Adjust the pattern to match your file name structure
matches = glob.glob(pattern)

matches.sort()

out = []

for event_files in matches[:-4]:
    out.append(pd.read_csv(event_files,sep='\t'))
all_metadata = pd.concat(out).reset_index(drop=True)

all_metadata


,onset,duration,trial_type,value,sample
0,5.410000,0.0,S01/Dynamic/0247_Right,12,6492
1,8.993333,0.0,S01/Dynamic/0247_Left,11,10792
2,12.526667,0.0,S01/Dynamic/0112_Left,5,15032
3,16.092500,0.0,S01/Dynamic/0022_Left,1,19311
4,19.625833,0.0,S01/Dynamic/0067_Left,3,23551
...,...,...,...,...,...
483,437.660000,0.0,S01/Dynamic/0112_Left,5,525192
484,441.193333,0.0,S01/Dynamic/0022_Right,2,529432
485,444.760000,0.0,S01/Dynamic/0157_Right,8,533712
486,448.325833,0.0,S01/Dynamic/0067_Left,3,537991


In [9]:
epochs_stacked.metadata = all_metadata

epochs_stacked.metadata

Adding metadata with 5 columns


,onset,duration,trial_type,value,sample
0,5.410000,0.0,S01/Dynamic/0247_Right,12,6492
1,8.993333,0.0,S01/Dynamic/0247_Left,11,10792
2,12.526667,0.0,S01/Dynamic/0112_Left,5,15032
3,16.092500,0.0,S01/Dynamic/0022_Left,1,19311
4,19.625833,0.0,S01/Dynamic/0067_Left,3,23551
...,...,...,...,...,...
483,437.660000,0.0,S01/Dynamic/0112_Left,5,525192
484,441.193333,0.0,S01/Dynamic/0022_Right,2,529432
485,444.760000,0.0,S01/Dynamic/0157_Right,8,533712
486,448.325833,0.0,S01/Dynamic/0067_Left,3,537991


In [10]:
block_type = [k.split('/')[1] for k in epochs_stacked.metadata.trial_type]
condition = [k.split('/')[2] for k in epochs_stacked.metadata.trial_type]

n_runs = 4
n_trials = 122

epochs_stacked.metadata['run_nr'] = np.repeat(np.arange(n_runs),n_trials)
epochs_stacked.metadata['block_type'] = block_type
epochs_stacked.metadata['condition'] = condition

epochs_stacked.metadata

,onset,duration,trial_type,value,sample,run_nr,block_type,condition
0,5.410000,0.0,S01/Dynamic/0247_Right,12,6492,0,Dynamic,0247_Right
1,8.993333,0.0,S01/Dynamic/0247_Left,11,10792,0,Dynamic,0247_Left
2,12.526667,0.0,S01/Dynamic/0112_Left,5,15032,0,Dynamic,0112_Left
3,16.092500,0.0,S01/Dynamic/0022_Left,1,19311,0,Dynamic,0022_Left
4,19.625833,0.0,S01/Dynamic/0067_Left,3,23551,0,Dynamic,0067_Left
...,...,...,...,...,...,...,...,...
483,437.660000,0.0,S01/Dynamic/0112_Left,5,525192,3,Dynamic,0112_Left
484,441.193333,0.0,S01/Dynamic/0022_Right,2,529432,3,Dynamic,0022_Right
485,444.760000,0.0,S01/Dynamic/0157_Right,8,533712,3,Dynamic,0157_Right
486,448.325833,0.0,S01/Dynamic/0067_Left,3,537991,3,Dynamic,0067_Left


In [11]:
import os

file_name = f'sub-{S}_Dynamic_preprocessed-epo.fif'
directory = f'{bids_top_dir}/derivatives/preprocessed/'
os.makedirs(directory, exist_ok=True)

# Save the file
epochs_stacked.save(f'{directory}{file_name}', overwrite=True)

